In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)

Queremos predecir los retrasos correspondientes con los 5 aeropuertos que experimentan mayor número de retrasos. En primer lugar debemos seleccionar dichos aeropuertos entre todos los seleccionados previamente

## Selección de aeropuertos

In [2]:
# Lectura de datos previamente limpiados
df = pd.read_parquet("../Preprocessing/flightsFilteredCleaned.parquet")
df['ORIGIN_AIRPORT'].unique()

array(['LAX', 'SFO', 'SEA', 'LAS', 'DEN', 'MSP', 'PHX', 'ORD', 'MCO',
       'BOS', 'DFW', 'IAH', 'ATL', 'EWR', 'CLT', 'DTW'], dtype=object)

In [3]:
# De todos estos aeropuertos seleccionamos los 5 con mayor numero de retrasos
# Agrupamos los datos de vuelos que experimentan retraso por aeropuertos
airports =  pd.DataFrame(df[df["ARRIVAL_DELAY"]>0].groupby(df['ORIGIN_AIRPORT'])["FLIGHT_NUMBER"].count()).reset_index()
airports = airports.rename(columns={"FLIGHT_NUMBER":"DELAYED_FLIGHTS"})

# Ordenamos los aeropuertos de tal manera que los 5 con mayor % de retrasos aparezcan los primeros
airports = airports.sort_values('DELAYED_FLIGHTS',ascending=False)
airports = airports.reset_index(level=0, drop=True)

airports.head()

,ORIGIN_AIRPORT,DELAYED_FLIGHTS
0,ATL,122213
1,ORD,121696
2,DFW,98565
3,DEN,86012
4,LAX,85590


In [4]:
# Seleccionamos dichos aeropuertos
top_5 = list(airports.loc[0:4,'ORIGIN_AIRPORT'])

# Filtramos nuestros datos para quedarnos únicamente con los correspondientes a dichos aeropuertos
df = df[df.ORIGIN_AIRPORT.isin(top_5)]

In [5]:
# Guardamos
df.to_parquet("top_5_delay.parquet", index=False)

## Preparamos los datos para el forecast

In [6]:
# Agrupamos los datos por aerolinea, fecha, y aeropuerto de origen y contamos el número de vuelos
data = pd.DataFrame(df[df["ARRIVAL_DELAY"]>0].groupby(["DATE","ORIGIN_AIRPORT"])["FLIGHT_NUMBER"].count()).reset_index()
data = data.rename(columns={"FLIGHT_NUMBER":"DELAYED_FLIGHTS"})
data.head()

,DATE,ORIGIN_AIRPORT,DELAYED_FLIGHTS
0,2015-01-01,ATL,162
1,2015-01-01,DEN,406
2,2015-01-01,DFW,453
3,2015-01-01,LAX,236
4,2015-01-01,ORD,271


Pueder ser que haya aeropuertos que no tengan retrasos algún día (caso raro). Para realizar el forecast necesitamos tener datos en todos los días de de todos los aeropuertos para todas las aerolíneas así que vamos a asegurarnos de que esto ocurra. Para ello crearemos un dataframe auxiliar con contenga todos los datos de fechas y aeropuertos, pero sin el número de vuelos retrasados

In [7]:
dates = [datetime(2015,1,1) + timedelta(days=d) for d in range((datetime(2015,12,31) - datetime(2015,1,1)).days + 1)]*5

airports = []
for airport in data['ORIGIN_AIRPORT'].unique():
    airport_dates=[airport]*365
    airports=airports+airport_dates

# Creamos el dataframe
aux = pd.DataFrame()
aux['DATE'] = dates
aux['ORIGIN_AIRPORT'] = airports

aux.head()

,DATE,ORIGIN_AIRPORT
0,2015-01-01,ATL
1,2015-01-02,ATL
2,2015-01-03,ATL
3,2015-01-04,ATL
4,2015-01-05,ATL


Una vez hecho esto vamos a unir el dataframe actual con el obtenido a partir del groupby, mediante un left join, de tal manera que si no tenemos datos de algún aeropuerto en alguna fecha exista una fila con valor nulo, el cual sustituiremos por cero

In [8]:
forecast_data = aux.merge(data, on=['DATE','ORIGIN_AIRPORT'], how='left')
forecast_data = forecast_data.fillna(0)
forecast_data['DELAYED_FLIGHTS'] = forecast_data['DELAYED_FLIGHTS'].astype(int)
forecast_data.head()

,DATE,ORIGIN_AIRPORT,DELAYED_FLIGHTS
0,2015-01-01,ATL,162
1,2015-01-02,ATL,445
2,2015-01-03,ATL,547
3,2015-01-04,ATL,653
4,2015-01-05,ATL,317


In [9]:
# Guardamos
forecast_data.to_parquet("forecast_data.parquet", index=False)